In [47]:
from os import environ
from os.path import join

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [48]:
api_url = "http://api.genius.com"
access_token = "Eaoa-I_DHC92LYuNAPbdh9o0-D-aXrl7T3aGUOOw-e6OrwymEoULjMgcVHC-1ZXS"
headers = {
    "Authorization": f"Bearer {access_token}"
}


In [53]:
# Get each artist url and ID with metadata in <head>
# meta[name="newrelic-resource-path"] on console
artists = {
    "%22Nekfeu%22": ("13063", "", {}, pd.DataFrame()),
    "Ninho": ("71286", "", {}, pd.DataFrame()),
    "%22Damso%22": ("45855", "", {}, pd.DataFrame()),
    "%22OrelSan%22": ("1286", "", {}, pd.DataFrame()),
    "%22SCH%22": ("276476", "", {}, pd.DataFrame()),
    "%22Oxmo Puccino%22": ("1947", "", {}, pd.DataFrame()),
    "%22Booba%22": ("1282", "", {}, pd.DataFrame()),
    "%22Vald%22": ("19217", "", {}, pd.DataFrame()),
    "%22Niska%22": ("234413", "", {}, pd.DataFrame()),
    "%22Akhenaton%22": ("1961", "", {}, pd.DataFrame())
}


In [62]:
# Get song name
def get_song_name(all_text: str):
    dash_index = all_text.find('–')
    pipe_index = all_text.find('|')
    song_name = all_text[dash_index + 1:pipe_index].strip()
    song_name = song_name.replace("'", "’")

    return song_name

# Remove additional info before lyrics
def delete_pre_lyrics(all_text: str, song_name: str):
    song_name_index = all_text.find(song_name)
    tmp = all_text[song_name_index + len(song_name):]   # Remove the song name at the beginning

    song_name_index = tmp.find(song_name)
    tmp3 = tmp[song_name_index:]
    tmp2 = tmp[song_name_index + len(song_name):]   # Remove everything before the second song name statement before the lyrics
    print(tmp)
    print(tmp2)
    print(tmp3)
    print()

    # pipe_index = tmp2.find('|')
    # lyrics_with_post = tmp2[pipe_index:]

    return tmp2

# Remove additional info after lyrics
def delete_post_lyrics(lyrics_with_post: str):
    end_index = lyrics_with_post.find("Embed")
    lyrics = lyrics_with_post[:end_index]
    lyrics = re.sub(r'\|\d+\|$', '', lyrics)

    return lyrics

# Remove all non-artist lyrics
def get_artist_lyrics(lyrics: str, artist: str):

    return lyrics

In [63]:
def get_lyrics(all_text: str, song_name: str, artist: str):
    print(artist)
    lyrics_with_post = delete_pre_lyrics(all_text, song_name)  # Remove additional info before lyrics
    lyrics = delete_post_lyrics(lyrics_with_post)  # Remove additional info after lyrics
    # cleaned_lyrics = get_artist_lyrics(lyrics, artist)    # Remove all non-artist lyrics

    return lyrics

In [64]:
for artist, value in artists.items():
    playlist = {}
    url = join(api_url, f"search?q={artist}")  # Form the url of the artist
    print(artist, ": ", url)

    r = requests.request("GET", url, headers=headers)  # Request to get max musics of the artist

    if r.status_code == 200:
        data = r.json()
        songs = data["response"]["hits"]  # Take the songs from the response
        
        if songs:
            songs_urls = [song["result"]["url"] for song in songs]  # Extract URLs for each song

            for song_url in songs_urls:
                song_response = requests.request("GET", song_url, headers=headers)  # Request to get the song page content
                if song_response.status_code == 200:
                    song_soup = BeautifulSoup(song_response.content, "html.parser") # Get the html of the page response
                    all_text = song_soup.get_text("|", strip=True)  # Retrieve all the text and concat with '|'
                    song_name = get_song_name(all_text)
                    lyrics = get_lyrics(all_text, song_name, artist)    # Retrieve the lyrics of the specified artist in the specified song's text
                    # print(all_text)
                    # print(lyrics)
                    # print()
                else:
                    print(f"Request for {song_url} failed !")
                playlist[song_name] = lyrics    # Add the song as key in the dictionary and the corresponding lyrics as value
            artists[artist] = (value[0], url, playlist, value[3])   # Add URL and playlist in the "artist" key of the "artists" dictionary
    else:
        print(f"Request for {artist} failed !")

%22Nekfeu%22 :  http://api.genius.com/search?q=%22Nekfeu%22
%22Nekfeu%22
 | Genius Lyrics|The Imperfect Journey|Featured|Charts|Videos|Promote Your Music|Sign Up|74|Humanoïde|Nekfeu|Track 1 on|Cyborg|Produced by|Loubenski|,|Népal|&|1|more|Un humanoïde est quelque-chose qui représente une personne « ressemblant à un humain ».

On remarque que ce titre est composé d'un grand nombre de questions, dans…|Read More|Dec. 1, 2016|1 viewer|710.3K views|173 Contributors|Humanoïde Lyrics|[Paroles de "Humanoïde"]|[Couplet 1]|Est-ce que tu t'es d'jà fait rabaisser par celle que t'aimais secrètement ?|Gentille en privé mais, d'vant les gens, cruelle et légère|Est-ce que t'as d'jà tapé quelqu'un juste pour qu'on te respecte ?|Sans excuse, est-ce que t'as d'jà regretté au point d'ber-ger ?|Est-ce que tu t'es d'jà dit : "Faut qu'j'me secoue, j'ai plus 16 ans" ?|N'être plus qu'une âme perdue, errer et rôder des heures|Voir un mec s'faire racketter, appeler au s'cours et pisser l'sang|Dans le même wagon 

In [61]:
# artists["%22Nekfeu%22"][2]
artists["Ninho"][2]
# artists["%22Damso%22"][2]
# artists["%22OrelSan%22"][2]
# artists["%22SCH%22"][2]
# artists["%22Oxmo Puccino%22"][2]
# artists["%22Booba%22"][2]
# artists["%22Vald%22"][2]
# artists["%22Niska%22"][2]
# artists["%22Akhenaton%22"][2]


{'Dis-moi que tu m’aimes Lyrics': '|[Paroles de "Dis-moi que tu m\'aimes"]|[Intro : tag]|Tonii on the Beats|[Couplet 1]|J\'avoue, j\'ai fait le con, enfermé entre quatre murs, trois heures du mat\', je repense à toi|Et si un jour, ça tire, est-ce que tu s\'ras mon armure ? Est-ce que t\'agiras comme une soldat ?|Y a toi, y a moi, y a nous, on ne fait plus qu\'un|Le plus important, c\'est l’atterrissage (Ouais, ouais, ouais), c\'est pas la chute|Et si je saute, tu s\'ras mon parachute (Eh),|on ira au Niagara voir les chutes d\'eau (D\'eau)|Des nuages à travers les hublots,|pourquoi j\'t\'ai pas rencontré plus tôt ?|[Pré-refrain]|T\'as c\'qu\'il faut là où il faut (Partout),|après toi, y a pas mieux (Bah ouais)|Dis-moi qu\'tu m\'aimes (Dis-moi qu\'tu m\'aimes) et regardes moi dans les yeux|[Refrain]|Dis-moi qu\'tu m\'aimes (Dis-moi qu\'tu m\'aimes)|Ouais, ouais, dis-moi qu\'tu m\'aime (Ouais, ouais, dis-moi qu\'tu m\'aimes)|J\'suis un voyou d\'la tess qui a besoin d\'amour, dis-moi qu\'t